In [5]:
import ipyvuetify as ipv      #    http://localhost:8888/voila/render/data_collect.ipynb
import ipywidgets as ipw
import pandas as pd
import numpy as np
from datetime import datetime
import time
from IPython.display import clear_output
import smtplib
from email.message import EmailMessage
import os
import base64
from openpyxl import load_workbook
import tempfile
import shutil
import threading


class downtime_app(object):

    def __init__(self):
        self.items = []  # Main data list
        self.temp_widgets = []  # Temporary input row widgets
        self.temp_form = None  # Container for temporary form
        self.start_time = None
        self.end_time = None
        self.hour_record = 7
        self.confirm_end_time = False
        self.operator_name = ""
        self.line_name = ""
        self.shift_val = ""
        self.create_widgets()
        threading.Thread(target=self.keep_alive, daemon=True).start()
        

    def create_widgets(self):
        # Buttons
        self.btn1 = ipv.Btn(color='primary', children=['Start Shift Collection'])
        self.btndown = ipv.Btn(color='red', children=['Line Down'])
        self.btnup = ipv.Btn(color='green', children=['Line Back Up']) 
        self.hourly = ipv.Btn(color='yellow', children=['Add Hourly Production'])
        self.btn5 = ipv.Btn(color='primary', children=['Shift Over'])
        self.btn6 = ipv.Btn(color='secondary', children=['Close Out Session']) 
        self.notice_box = ipw.HTML(value="",layout=ipw.Layout(align_self='center', visibility='hidden', height='30px'))

        self.hour_list_1 = ['6:00 - 7:00','7:00 - 8:00','8:00 - 9:00','9:00 - 10:00','10:00 - 11:00','11:00 - 12:00','12:00 - 1:00','1:00 - 2:00']
        self.hour_list_2 = ['2:00 - 3:00','3:00 - 4:00','4:00 - 5:00','5:00 - 6:00','6:00 - 7:00','7:00 - 8:00','8:00 - 9:00','9:00 - 10:00']

        # Data Table
        self.headers = [
            {'text': 'Time', 'value': 'Time', 'style': 'width: 100px;'},
            {'text': 'Status', 'value': 'Status', 'style': 'width: 120px;'},
            {'text': 'Location', 'value': 'Location', 'style': 'width: 150px;'},
            {'text': 'Cause', 'value': 'Cause', 'style': 'width: 200px;'},
            {'text': 'Approx. Units Lost', 'value': 'Units Lost', 'style': 'width: 160px;'},
            {'text': 'Notes', 'value': 'Notes', 'style': 'width: 250px;'}
        ]

        self.dt = ipv.DataTable(headers=self.headers,items=self.items,items_per_page=10,class_="elevation-1",
                                show_select=False)  

        # Button event connections
        self.btn1.on_event('click', self.add_start_time)
        self.btndown.on_event('click', self.show_temp_form_down)
        self.btnup.on_event('click', self.show_temp_form_up)
        self.hourly.on_event('click', self.show_temp_hourly_rate)
        self.btn5.on_event('click', self.add_end_time)
        self.btn6.on_event('click', self.close_session)
        self.btndown.disabled = True
        self.btnup.disabled = True
        self.btn5.disabled = True
        self.hourly.disabled = True

        # Temporary form placeholder (hidden initially)
        self.temp_form_container = ipw.VBox([])

        # Layout
        self.button_row_1 = ipw.HBox([
            self.btn1,
            ipw.Layout(width='100%'),  # spacer
            self.btn6
        ], layout=ipw.Layout(justify_content='space-between'))

        self.button_row_2 = ipw.HBox(
            [self.btn5],layout=ipw.Layout(margin='10px 0px'))

        spacer = ipw.HTML(value='  ')
        self.button_row_3 = ipw.HBox(
            [self.btndown, self.btnup,spacer,self.hourly],
            layout=ipw.Layout(margin='10px 0px')
        )

        logo = ipw.Image(value=open("logo.png", "rb").read(),format='png',layout=ipw.Layout(height='90px'))
        title = ipw.HTML(value="<h2 style='margin: 0; padding-left: 12px;'>Production Downtime Logger</h2>")
        header = ipw.HBox([logo, title], layout=ipw.Layout(align_items='center', padding='10px 0'))

        self.form = ipw.VBox([
            header,
            self.button_row_1,
            self.button_row_2,
            self.notice_box,
            self.button_row_3,
            self.temp_form_container,
            self.dt
        ])

    def prompt_user_info(self):
        name_input = ipw.Text(
            description="Operator:",
            placeholder="Enter your name",
            layout=ipw.Layout(width='300px')  # fixed width
        )

        shift_dropdown = ipw.Dropdown(
            description="Shift:",
            options=['Shift 1','Shift 2'],
            value='Shift 1',
            layout=ipw.Layout(width='300px')  # match width
        )

        line_dropdown = ipw.Dropdown(
            description="Line:",
            options=['Line 1', 'Line 3', 'Line 4','Syrups','Other'],
            value='Syrups',
            layout=ipw.Layout(width='300px')  # match width
        )

        custom_line_input = ipw.Text(
            description="Custom Line",
            layout=ipw.Layout(display='none', width='300px')
        )

        # Show/hide custom line input if "Other" is selected
        def toggle_custom_line(change):
            if change['new'] == 'Other':
                custom_line_input.layout.display = 'block'
            else:
                custom_line_input.layout.display = 'none'

        line_dropdown.observe(toggle_custom_line, names='value')

        submit_button = ipv.Btn(
            color='success',
            children=['Submit'],
            layout=ipw.Layout(width='300px')  # match input width
        )

        def handle_submit(widget, event, data):
            name_val = name_input.value.strip()
            if line_dropdown.value == 'Other':
                line_val = custom_line_input.value.strip()
            else:
                line_val = line_dropdown.value

            if not name_val or not line_val:
                self.show_notice("⚠️ Please enter both your name and line.")
                return

            self.operator_name = name_val
            self.line_name = line_val
            self.shift_val = shift_dropdown.value
            if self.shift_val == 'Shift 1':
                self.hour_list = self.hour_list_1
            elif self.shift_val == 'Shift 2':
                self.hour_list = self.hour_list_2
            
            self.temp_form_container.children = []  # Clear form
            self._record_start_time()

        submit_button.on_event('click', handle_submit)

        # Layout block
        self.temp_form_container.children = [
            ipw.VBox([
                name_input,
                line_dropdown,
                shift_dropdown,
                custom_line_input,
                submit_button
            ])
        ]


    def show_notice(self, message, duration=3):
        self.notice_box.value = f"""<div style='color: black; background-color: #fff176; padding: 5px 20px; border-radius: 6px; text-align: center; font-weight: bold;'>{message}</div>"""
        self.notice_box.layout.visibility = 'visible'

        # Cancel existing timer if running
        if hasattr(self, 'notice_timer') and self.notice_timer is not None:
            self.notice_timer.cancel()

        def hide_notice():
            self.notice_box.layout.visibility = 'hidden'

        self.notice_timer = threading.Timer(duration, hide_notice)
        self.notice_timer.start()

    def open_hourly_entry(self, *args):
        if not self.start_time or not self.line_name:
            self.show_notice("⚠️ Please set the shift start time and line name before entering hourly data.", duration=5)
            return

        # Build the local path
        date_str = self.start_time.strftime('%m-%d-%Y')
        line_clean = self.line_name.replace(" ", "_")
        shift_clean = self.shift_val.replace(" ", "_")
        filename = f"Local_{line_clean}_{shift_clean}_{date_str}.xlsx"
        local_path = os.path.join(os.getcwd(), filename)

        # Debug: Log file path
        print(f"Checking for hourly data at: {local_path}")

        if os.path.exists(local_path):
            try:
                df_hourly = pd.read_excel(local_path, sheet_name="Hourly Output")

                if not df_hourly.empty:
                    self.hours = df_hourly.get("Hour Range", []).tolist()
                    self.results = df_hourly.get("Bottles Completed", []).tolist()
                    self.hour_downtime = df_hourly.get("Downtime Minutes", []).tolist()
                    self.hourly_notes = df_hourly.get("Notes", []).tolist()
                    self.hour_record = 7 + len(self.results)
                    self.show_notice(f"✅ Restored {len(self.hours)} hourly records from file.", duration=4)
                else:
                    self.show_notice("ℹ️ Hourly Output sheet is empty.", duration=4)
                    self.hours = []
                    self.results = []
                    self.hour_downtime = []
                    self.hourly_notes = []


            except Exception as e:
                self.show_notice(f"⚠️ Couldn't load hourly data: {e}", duration=5)
                print(f"Hourly load error: {e}")
        else:
            self.show_notice("ℹ️ No previous hourly data file found.", duration=4)
            # You may still want to clear memory here
            self.hours = []
            self.results = []
            self.hour_downtime = []
            self.hourly_notes = []


    def show_temp_hourly_rate(self, *args):
        self.temp_widgets = []
        self.add_btn_temp = ipv.Btn(color='green', children=['Add Data'],layout=ipw.Layout(width='200px'))
        self.add_btn_temp.on_event('click', self.add_hour)
        self.cancel_btn_temp = ipv.Btn(color='red', children=['Cancel and Clear'],layout=ipw.Layout(width='200px'))
        self.cancel_btn_temp.on_event('click', self.cancel_hour)
        self.button_row = ipw.HBox(
            [self.add_btn_temp, self.cancel_btn_temp],
            layout=ipw.Layout(margin='10px 0px')
        )
        self.output = ipw.Text(description='Bottle Output', value='', disabled=False, layout=ipw.Layout(width='400px'))
        self.down_min = ipw.Text(description='Downtime(m)', value='', disabled=False, layout=ipw.Layout(width='400px'))
        self.hour_note = ipw.Text(description='Notes', value='', layout=ipw.Layout(width='400px'))
        self.hour_range = ipw.Text(description='Hour', value='', disabled=True, layout=ipw.Layout(width='400px'))
        i = self.hour_record - 7
        self.hour_range.value = self.hour_list[i]
        
        self.temp_widgets = [self.hour_range,self.output,self.down_min,self.hour_note,self.button_row]
        
        self.temp_form_container.children = self.temp_widgets
        self.holder = 0
        if self.btnup.disabled == False:
            self.holder = 1
            self.btnup.disabled = True

        elif self.btndown.disabled == False:
            self.holder = 2
            self.btndown.disabled = True
        self.hourly.disabled = True

    def add_hour(self, *args):
        if self.output.value == '' or self.down_min.value == '':
            self.show_notice("⚠️ Please add both downtime and completed bottles", duration=4)
            return

        self.results.append(self.output.value)
        self.hourly_notes.append(self.hour_note.value)
        self.hour_downtime.append(self.down_min.value)
        self.hours.append(self.hour_list[self.hour_record - 7])

        self.hour_record += 1
        if self.hour_record == 15:
            self.show_notice("⚠️ Hourly Bottle Rate Complete", duration=3)
            self.hourly.disabled = True
            if self.holder == 1:
                self.btnup.disabled = False
            elif self.holder == 2:
                self.btndown.disabled = False
            self.save_to_excel_live()
            self.temp_widgets = []
            self.temp_form_container.children = []
            return
        
        # Clear the form and disable buttons
        self.temp_widgets = []
        self.temp_form_container.children = []
        self.show_notice("✅ Data for last hour Successfully Added!", duration=4)
        self.save_to_excel_live()
        if self.holder == 1:
            self.btnup.disabled = False
        elif self.holder == 2:
            self.btndown.disabled = False
        self.hourly.disabled = False

    def cancel_hour(self, *args):
        self.temp_widgets = []
        self.temp_form_container.children = []
        self.show_notice("Data for last hour Successfully Cancelled", duration=4)
        if self.holder == 1:
            self.btnup.disabled = False
        elif self.holder == 2:
            self.btndown.disabled = False
        self.hourly.disabled = False


    def add_start_time(self, *args):
        # Prompt for name/line if not already entered
        if not self.operator_name or not self.line_name:
            self.prompt_user_info()
            return
        self._record_start_time()

    def _record_start_time(self):
        current_time = datetime.now()
        self.start_time = current_time
        new_item = {
            'Time': current_time.strftime('%H:%M:%S'),'Status': 'Shift Started',
            'Location': '', 'Cause': '', 'Units Lost': '', 'Notes': ''
        }
        self.items = self.items + [new_item]  # ✅ Triggers table refresh properly
        self.dt.items = self.items.copy()     # ✅ Important for ipyvuetify

        self.btn1.disabled = True
        self.btndown.disabled = False
        self.btn5.disabled = False
        self.hourly.disabled = False
        self.show_notice("Start time recorded.")
        self.save_to_excel_live()
        self.open_hourly_entry()


    def add_end_time(self, *args):

        if not self.confirm_end_time:
            self.confirm_end_time = True
            self.show_notice("⚠️ Are you sure you want to end data collection? . Click again to confirm.", duration=4)

            import threading
            def reset_confirmation():
                self.confirm_end_time = False
            threading.Timer(10, reset_confirmation).start()
            return

        # Proceed with end time recording
        current_time = datetime.now()
        self.end_time = current_time
        self.items.append({
            'Time': datetime.now().strftime('%H:%M:%S'),'Status': 'Shift Over',
            'Location': '', 'Cause': '', 'Units Lost': '', 'Notes': ''
                })
        self.dt.items = list(self.items)
        self.show_notice("✅ End time recorded.")
        self.btndown.disabled = True
        self.btnup.disabled = True
        self.btn5.disabled = True
        self.confirm_end_time = False  # Reset confirmation state
        self.save_to_excel_live()


    def show_temp_form_down(self, *args):
        if self.start_time is None:
            self.show_notice("Record a start time to begin adding events", duration=4)
            return

        current_time = datetime.now().strftime('%H:%M:%S')
        self.btnup.disabled = False

        location_options = ['Depal', 'Air Rinser', 'Filler Capper', 'Sleever', 'Unscrambler','Lunch','Break', 'Other - Describe in Notes']

        causes_map = {
            'Depal': ['Cause D1', 'Cause D2', 'Cause D3'],
            'Air Rinser': ['Cause A1', 'Cause A2', 'Cause A3'],
            'Filler Capper': ['Cause F1', 'Cause F2', 'Cause F3'],
            'Sleever': ['Cause S1', 'Cause S2', 'Cause S3'],
            'Unscrambler': ['Cause U1', 'Cause U2', 'Cause U3']
        }

        self.notes_text = ipw.Text(
            description='Notes',
            layout=ipw.Layout(width='400px', description_width='200px')
            )

        self.location_dropdown = ipw.Dropdown(
            options=location_options,
            description='Location',
            value=location_options[0],
            layout=ipw.Layout(width='400px', description_width='200px')
            )

        self.cause_dropdown = ipw.Dropdown(
            options=causes_map.get(location_options[0], []),
            description='Cause',
            layout=ipw.Layout(width='400px', description_width='200px')
            )

        def on_location_change(change):
            loc = change['new']
            if loc == 'Other - Describe in Notes' or loc == 'Lunch' or loc == 'Break':
                self.cause_dropdown.layout.display = 'none'
                self.notes_text.description = 'Notes'
            else:
                self.cause_dropdown.options = causes_map.get(loc, [])
                self.cause_dropdown.layout.display = 'block'
                self.cause_dropdown.value = causes_map.get(loc, [None])[0]
                self.notes_text.description = 'Notes'

        self.location_dropdown.observe(on_location_change, names='value')
        self.add_btn_temp = ipv.Btn(color='green', children=['Add Event'],layout=ipw.Layout(width='200px'))
        self.add_btn_temp.on_event('click', self.add_temp_to_main_down)
        self.cancel_btn_temp = ipv.Btn(color='red', children=['Cancel and Clear'],layout=ipw.Layout(width='200px'))
        self.cancel_btn_temp.on_event('click', self.cancel_down)
        self.button_row = ipw.HBox(
            [self.add_btn_temp, self.cancel_btn_temp],
            layout=ipw.Layout(margin='10px 0px')
        )

        self.temp_widgets = [
            ipw.Text(description='Time', value=current_time, disabled=True, layout=ipw.Layout(width='150px')),
            self.location_dropdown,
            self.cause_dropdown,
            ipw.Text(description='Units Lost', layout=ipw.Layout(width='400px')),
            self.notes_text,
            self.button_row
        ]

        self.temp_form_container.children = self.temp_widgets
        self.btndown.disabled = True
        self.btnup.disabled = True
        self.hourly.disabled = True


    def cancel_down(self, *args):
        self.temp_widgets = []
        self.temp_form_container.children = []
        self.btndown.disabled = False
        self.btnup.disabled = True
        self.hourly.disabled = False
        self.show_notice("Event Successfully Cancelled", duration=4)


    def add_temp_to_main_down(self, *args):
        if not self.temp_widgets:
            self.show_notice("⚠️ No event to add.", duration=3)
            return

        selected_location = self.location_dropdown.value
        location_value = selected_location  # no custom input now

        # For cause: only use cause_dropdown value if location != 'Other', else no cause
        if selected_location == 'Other - Describe in Notes':
            cause_value = ''  # no cause dropdown for Other
        else:
            cause_value = self.cause_dropdown.value

        new_item = {
            'Time': self.temp_widgets[0].value,          # Time text widget index 0
            'Location': location_value,                   # Location dropdown value
            'Status': 'Line Down',
            'Cause': cause_value,                         # cause dropdown or empty string
            'Units Lost': self.temp_widgets[3].value,    # Units Lost text widget index 4
            'Notes': self.temp_widgets[4].value    # Notes text widget index 5
        }

        self.items.append(new_item)
        self.dt.items = list(self.items)

        # Clear the form and disable buttons
        self.temp_widgets = []
        self.temp_form_container.children = []
        self.btndown.disabled = True
        self.btnup.disabled = False
        self.hourly.disabled = False
        self.show_notice("✅ Event Successfully Added!", duration=4)

        self.save_to_excel_live()


    def show_temp_form_up(self, *args):
        self.show_notice("⚠️ Add Troubleshooting Details in Notes Selection", duration=5)
        current_time = datetime.now().strftime('%H:%M:%S')

        self.notes_text = ipw.Text(
            description='Notes',
            layout=ipw.Layout(width='400px', description_width='200px')
            )

        self.add_btn_temp = ipv.Btn(color='green', children=['Add Event'],layout=ipw.Layout(width='200px'))
        self.add_btn_temp.on_event('click', self.add_temp_to_main_up)
        self.cancel_btn_temp = ipv.Btn(color='red', children=['Cancel and Clear'],layout=ipw.Layout(width='200px'))
        self.cancel_btn_temp.on_event('click', self.cancel_up)
        self.button_row = ipw.HBox(
            [self.add_btn_temp, self.cancel_btn_temp],
            layout=ipw.Layout(margin='10px 0px')
        )

        self.temp_widgets = [
            ipw.Text(description='Time', value=current_time, disabled=True, layout=ipw.Layout(width='150px')),
            self.notes_text,
            self.button_row
        ]

        self.temp_form_container.children = self.temp_widgets
        self.btnup.disabled = True
        self.btndown.disabled = True
        self.hourly.disabled = True

    def cancel_up(self, *args):
        self.temp_widgets = []
        self.temp_form_container.children = []
        self.btndown.disabled = True
        self.btnup.disabled = False
        self.hourly.disabled = False
        self.show_notice("Event Successfully Cancelled", duration=4)
        

    def add_temp_to_main_up(self, *args):
        new_item = {
            'Time': self.temp_widgets[0].value,          # Time text widget index 0
            'Location': '',                   # Location dropdown value
            'Status': 'Line Back Up',
            'Cause': '',                         # cause dropdown or empty string
            'Units Lost': '',    # Units Lost text widget index 4
            'Notes': self.temp_widgets[1].value    # Notes text widget index 5
        }

        self.items.append(new_item)
        self.dt.items = list(self.items)

        # Clear the form and disable buttons
        self.temp_widgets = []
        self.temp_form_container.children = []
        self.btndown.disabled = False
        self.btnup.disabled = True
        self.hourly.disabled = False
        self.show_notice("✅ Event Successfully Added!", duration=4)

        self.save_to_excel_live()

    def save_to_excel_live(self):
        if not self.start_time or not self.operator_name or not self.line_name:
            return
        df = pd.DataFrame(self.items)

        # ✅ Local file keeps full date
        date_str = self.start_time.strftime('%m-%d-%Y')
        line_clean = self.line_name.replace(" ", "_")
        shift_clean = self.shift_val.replace(" ", "_")
        filename = f"Local_{line_clean}_{shift_clean}_{date_str}.xlsx"
        local_excel_path = os.path.join(os.getcwd(), filename)

        # ✅ Teams file removes year to differentiate
        month_day_str = self.start_time.strftime('%m-%d')
        teams_filename = f"Live_{line_clean}_{shift_clean}_{month_day_str}.xlsx"
        teams_excel_path = os.path.join(r"C:\Users\Drew.StLouis\Heartland FPG\Liquids CI - Team\12 Downtime Files",teams_filename)

        # Header row
        header_row = {
            'Time': 'Operator:',
            'Status': self.operator_name,
            'Location': '',
            'Cause': '',
            'Units Lost': '',
            'Notes': ''
        }

        # This helper handles the save process for a given path
        def save_to_path(path):
            try:
                if os.path.exists(path):
                    existing_df = pd.read_excel(path)

                    # Check if header exists
                    if not existing_df.empty and str(existing_df.iloc[0]['Time']).strip() == 'Operator:':
                        existing_df_data = existing_df.iloc[1:]  # Skip the header row
                        df_to_add = df[~df['Time'].isin(existing_df_data['Time'])]
                        combined_df = pd.concat([existing_df, df_to_add], ignore_index=True)
                    else:
                        header_df = pd.DataFrame([header_row])
                        combined_df = pd.concat([header_df, existing_df, df], ignore_index=True)
                else:
                    header_df = pd.DataFrame([header_row])
                    combined_df = pd.concat([header_df, df], ignore_index=True)

                # Save to a temporary file first (safest way)
                with tempfile.NamedTemporaryFile(delete=False, suffix=".xlsx") as tmp:
                    temp_path = tmp.name
                combined_df.to_excel(temp_path, index=False)

                # Add hourly output sheet
                hourly_df = pd.DataFrame({
                    "Hour Range": self.hours,
                    "Bottles Completed": self.results,
                    "Downtime Minutes": self.hour_downtime,
                    "Notes": self.hourly_notes
                })
                with pd.ExcelWriter(temp_path, engine="openpyxl", mode="a", if_sheet_exists="overlay") as writer:
                    hourly_df.to_excel(writer, sheet_name="Hourly Output", index=False)

                # Move or overwrite final
                try:
                    shutil.move(temp_path, path)
                except PermissionError:
                    os.remove(temp_path)
                    if "Teams" in path:
                        self.show_notice("⚠️ Teams Excel is open — please close it to update.", duration=5)

            except Exception as e:
                self.show_notice(f"❌ Excel update error ({'Teams' if 'Teams' in path else 'Local'}): {e}", duration=5)

        # Save to both locations
        save_to_path(teams_excel_path)
        save_to_path(local_excel_path)


    def close_session(self, *args):
        if self.end_time is None:
            self.show_notice("⚠️ End Time not set. Click [Shift Over] to proceed.", duration=4)
            return

        # ✅ Require exactly 8 hourly records
        if self.hour_record != 15:
            self.show_notice(f"⚠️ All 8 hourly records required to close session. You have {self.hour_record - 7}.", duration=5)
            return

        # ✅ Construct the local filename (same logic as save_to_excel_live)
        date_str = self.start_time.strftime('%m-%d-%Y')
        line_clean = self.line_name.replace(" ", "_")
        shift_clean = self.shift_val.replace(" ", "_")
        filename = f"Local_{line_clean}_{shift_clean}_{date_str}.xlsx"
        filepath = os.path.join(os.getcwd(), filename)

        self.btn6.disabled = True
        self.hourly.disabled = True
        self.show_notice("✅ Shift Completed, Close out tab or refresh to proceed")

    def keep_alive(self):
        while True:
            try:
                self.save_to_excel_live()  # saves both files
            except Exception as e:
                print(f"Keep alive save error: {e}")
            time.sleep(180)  # sleep 3 minutes

In [6]:
t = downtime_app()
t.form